# Fitting Signal Detection Measures to ROC Curves

In [201]:
import numpy as np
from scipy import stats
from scipy.optimize import minimize
from sklearn.utils import check_consistent_length
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
np.seterr(all='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

## Main Functions
[1] Robert R. Sokal and F. James Rohlf. Biometry: The Principles and Practices of Statistics in BiologicalResearch. W.H. Freeman, 3 edition, 1994  
[2] Hoey, J. (2012). The Two-Way Likelihood Ratio (G) Test and Comparison to Two-Way Chi Squared Test.

In [241]:
def compute_dprime(tp, fp):
    # untested
    return stats.norm.ppf(tp) - stats.norm.ppf(fp)

def compute_c_bias(tp, fp):
    # untested
    return - 0.5 * (stats.norm.ppf(tp) + stats.norm.ppf(fp))

def accumulate(*arrays):
    # untested
    out = []
    for a in arrays:
        accumulated = np.cumsum(a)
        out.append(accumulated)
    return out

def to_freq(*arrays):
    # untested
    out = []
    for a in arrays:
        converted = np.array([(x + i / len(a)) / (max(a)+1) for i, x in enumerate(a, start=1)])
        out.append(converted)
    return out

def convert_to_roc(a, b):
    # untested
    a_c, b_c = accumulate(a, b)
    a_f, b_f = to_freq(a_c, b_c)
    return a_f, b_f

def g_test(x, x_freq, expected, x_max):
    # untested
    # Refs: [1, 2]
    # Two-way log-likelihood G-test
    # Implementation issues:
    #    depending on minimization starting variables, the log expressions can throw errors from negative values and div/0 etc. Numpy just warns and continues.
    a = 2 * x * np.log(x_freq / expected) 
    b = 2 * (x_max - x) * np.log((1 - x_freq) / (1 - expected))
    return a + b

def threshold_model_expectation(R, noise_frequency):
    # untested
    # Get the expected values for signals and noises
    expected_signal = ((1 - R) * noise_frequency + R)[:-1]
    expected_noise = noise_frequency[:-1]
    return expected_signal, expected_noise

def signal_detection_model_expectation(d, c):
    # untested
    expected_signal = stats.norm.cdf(d / 2 - c)
    expected_noise = stats.norm.cdf(-d / 2 - c)
    return expected_signal, expected_noise

def dual_process_model_expectation(R, d, c):
    # untested
    expected_signal = R + (1 - R) * stats.norm.cdf(d / 2 - c)
    expected_noise = stats.norm.cdf(-d / 2 - c)
    return expected_signal, expected_noise

def detection_model(parameters, labels, signal, noise, model='sdt'):
    # untested
    # model options: 'threshold', 'sdt', 'dpsdt', 'thresh'
    
    # Implementation issues:
    #    parameters and labels are specified as separate ndarrays arrays because of how the optimizer works. Maybe this can be improved?
    
    #------------------------- Check Inputs ---------------------------#
    # Ensure equal length
    check_consistent_length(signal, noise)
    check_consistent_length(parameters, labels)
    
    # Accumulate (c) the signal and noise data
    signal_c, noise_c = accumulate(signal, noise) # np.cumsum(signal), np.cumsum(noise)
    # Get frequencies for the cumulative signal and noise
    signal_f, noise_f = to_freq(signal_c, noise_c)
    
    #------------------------- Model Specific -------------------------#
    if model == 'sdt':
        if 'd' not in labels:
            raise ValueError(f"Either or both of `R` and `d` not specified in parameter labels. labels specified were: {labels}.")
        d = parameters[labels == 'd'].item()
        c = parameters[(labels!='R') & (labels!='d')] # Can c_values be 0? what about 1?
        exp_signal, exp_noise = signal_detection_model_expectation(d, c)
        
    elif model == 'dpsdt':
        if ('R' not in labels) or ('d' not in labels):
            raise ValueError(f"Either or both of `R` and `d` not specified in parameter labels. labels specified were: {labels}.")
        # Grab the R and d values
        R = parameters[labels == 'R'].item()
        d = parameters[labels == 'd'].item()
    
        c = parameters[(labels!='R') & (labels!='d')] # Can c_values be 0? what about 1?
        # Get the expected values for signals and noises
        exp_signal, exp_noise = dual_process_model_expectation(R, d, c)
    
    elif model == 'thresh':
        if ('R' not in labels):
            raise ValueError(f"Either or both of `R` and `d` not specified in parameter labels. labels specified were: {labels}.")
        R = parameters[labels == 'R'].item()
        exp_signal, exp_noise = threshold_model_expectation(R, noise_f)
    #------------------------------------------------------------------#

    # Compute the gsquared using the dual process model parameters
    #   Still getting runtime warnings about division. Function only works with numpy, so can't use math.
    signal_gsquared = g_test(x=signal_c[:-1], x_freq=signal_f[:-1], expected=exp_signal, x_max=signal_c.max())
    noise_gsquared = g_test(x=noise_c[:-1], x_freq=noise_f[:-1], expected=exp_noise, x_max=noise_c.max())
    
    g_squared_summed = np.sum([signal_gsquared, noise_gsquared])
    
    return g_squared_summed


def optimize_model(objective, labels, signal, noise, model, iterations=25, parameters=None, verbose=True):
    for i in range(iterations):
        try:
            if not parameters:
                # Random parameter initialization (default)
                parameters = np.random.uniform(-.5, .5, len(labels))
                
            opt = minimize(fun=objective, x0=parameters, args=(labels, signal, noise, model), tol=1e-4)
            if opt.success:
                # Converged at (local?) minimum.
                if verbose==True:
                    print(opt)
                else:
                    print("Optimisation successful\n")
                break
            else:
                raise ValueError("Optimisation unsuccessful")
        except:
            opt = None
    return opt

## Testing the different models with example data

In [205]:
# Example data
signal = [508,224,172,135,119,63] # Signal present examples
noise = [102,161,288,472,492,308] # Signal absent examples

### 1.1. signal detection model

In [206]:
# Arguments
# Parameters randomly set in the fit_model function
parameters = np.array([1.12279662339189, 0.871166834070118, 0.405779553457989, -0.0588639712751784, -0.644343940682155, -1.42214656169388])
labels = np.array(['d', 'c1', 'c2', 'c3', 'c4', 'c5'])
expected = 91.8424069319085

model_fit = detection_model(parameters=parameters, labels=labels, signal=signal, noise=noise, model='sdt')

np.allclose(model_fit, expected)
print(f"expected G\N{SUPERSCRIPT TWO}: {expected}\nfound G\N{SUPERSCRIPT TWO}:{model_fit}")

expected G²: 91.8424069319085
found G²:91.84240693190836


#### 1.2. Optimizing signal detection model

In [207]:
opt = optimize_model(objective=detection_model, labels=labels, signal=signal, noise=noise,model='sdt')

if opt:
    if opt.success:
        optimal_parameters = {l:v for l, v in zip(labels, opt.x)}
        print(f"optimal parameters found:\n{optimal_parameters}")
else:
    print("Failure.")

      fun: 91.8424069266944
 hess_inv: array([[ 2.91125674e-04,  3.00533900e-05, -1.29080022e-06,
        -3.06755184e-05, -5.65082130e-05, -8.67297607e-05],
       [ 3.00533900e-05,  3.89386897e-04, -2.84210799e-06,
        -4.88970974e-06, -2.71882648e-08, -4.62776898e-06],
       [-1.29080022e-06, -2.84210799e-06,  2.88876527e-04,
        -1.53119583e-06, -1.35947970e-05,  6.96552166e-06],
       [-3.06755184e-05, -4.88970974e-06, -1.53119583e-06,
         2.70395950e-04,  6.71287497e-06,  1.03260707e-05],
       [-5.65082130e-05, -2.71882648e-08, -1.35947970e-05,
         6.71287497e-06,  2.96333906e-04,  1.47781556e-05],
       [-8.67297607e-05, -4.62776898e-06,  6.96552166e-06,
         1.03260707e-05,  1.47781556e-05,  4.78866801e-04]])
      jac: array([-7.91549683e-05,  2.95639038e-05, -5.24520874e-05,  3.05175781e-05,
       -2.86102295e-05,  3.33786011e-05])
  message: 'Optimization terminated successfully.'
     nfev: 245
      nit: 17
     njev: 30
   status: 0
  success: 

### 2.1. dual process model

In [208]:
parameters = np.array([0.329619769943936, 0.669264105874664, 1.30349820023126, 0.683947212600267, 0.145604921894773, -0.473844533479801, -1.25079084398587])
labels = np.array(['R', 'd', 'c1', 'c2', 'c3', 'c4', 'c5'])
expected = 22.6109004733713

model_fit = detection_model(parameters=parameters, labels=labels, signal=signal, noise=noise, model='dpsdt')

np.allclose(model_fit, expected)
print(f"expected G\N{SUPERSCRIPT TWO}: {expected}\nfound G\N{SUPERSCRIPT TWO}:{model_fit}")

expected G²: 22.6109004733713
found G²:22.61090047337164


#### 2.2. Optimizing dual-process model

In [232]:
opt = optimize_model(objective=detection_model, labels=labels, signal=signal, noise=noise, iterations=500, model='dpsdt')

if opt:
    if opt.success:
        optimal_parameters = {l:v for l, v in zip(labels, opt.x)}
        print(f"optimal parameters found:\n{optimal_parameters}")
else:
    print("Failure.")

      fun: 22.610900467600636
 hess_inv: array([[ 0.00034061, -0.00064488,  0.00053924,  0.00041126,  0.00031524,
         0.00025528,  0.00024294],
       [-0.00064488,  0.00170723, -0.00112455, -0.00087625, -0.00068964,
        -0.0006099 , -0.00060252],
       [ 0.00053924, -0.00112455,  0.00185877,  0.00064056,  0.00052529,
         0.00040842,  0.00041313],
       [ 0.00041126, -0.00087625,  0.00064056,  0.00099473,  0.00039944,
         0.00034665,  0.00033083],
       [ 0.00031524, -0.00068964,  0.00052529,  0.00039944,  0.00069496,
         0.00026026,  0.0002439 ],
       [ 0.00025528, -0.0006099 ,  0.00040842,  0.00034665,  0.00026026,
         0.0005609 ,  0.00021259],
       [ 0.00024294, -0.00060252,  0.00041313,  0.00033083,  0.0002439 ,
         0.00021259,  0.00072682]])
      jac: array([-2.00271606e-05,  8.82148743e-06, -5.34057617e-05, -1.66893005e-05,
       -2.38418579e-06, -1.28746033e-05,  0.00000000e+00])
  message: 'Optimization terminated successfully.'
     n

### 3.1. threshold model

In [234]:
parameters = np.array([0.520274590415736])
labels = np.array(['R'])
expected = 163.275256078908
model_fit = detection_model(parameters=parameters, labels=labels, signal=signal, noise=noise, model='thresh')

np.allclose(model_fit, expected)
print(f"expected G\N{SUPERSCRIPT TWO}: {expected}\nfound G\N{SUPERSCRIPT TWO}:{model_fit}")

expected G²: 163.275256078908
found G²:163.27525607890783


#### 3.2. Optimizing threshold model

This one gives errors and needs more work. It does work if you run it enough times.

In [240]:
opt = optimize_model(objective=detection_model, labels=labels, signal=signal, noise=noise, model='thresh')

if opt:
    if opt.success:
        optimal_parameters = {l:v for l, v in zip(labels, opt.x)}
        print(f"optimal parameters found:\n{optimal_parameters}")
else:
    print("Failure.")

163.27525607888896
      fun: 163.27525607888896
 hess_inv: array([[4.14264526e-05]])
      jac: array([-7.62939453e-06])
  message: 'Optimization terminated successfully.'
     nfev: 21
      nit: 5
     njev: 7
   status: 0
  success: True
        x: array([0.52027462])
optimal parameters found:
{'R': 0.5202746232698504}
